In [2]:
import pandas as pd

In [3]:
## Load raw data and combine cost and mean dynamics parameters into same dataframe while cleaning up column names

df_raw_costs = pd.read_csv('./raw_data/components_costs_raw.csv')
df_raw_dynamics = pd.read_csv('./raw_data/components_dynamics_raw.csv')

# assign a unique index to each row in df_raw_costs as first column
df_raw_costs['component_id'] = df_raw_costs.index
cols = df_raw_costs.columns.tolist()
cols = cols[-1:] + cols[:-1]
df_raw_costs = df_raw_costs[cols]

# change column names
df_raw_costs = df_raw_costs.rename(columns={'Uniformat Code': 'uniformat_code'})
df_raw_costs = df_raw_costs.rename(columns={'Replacement Cost': 'replacement_cost'})
df_raw_costs = df_raw_costs.rename(columns={'Importance Index': 'importance_index'})
df_raw_costs = df_raw_costs.rename(columns={'Component': 'component_type_name'})


# change df_raw_dynamics column names
df_raw_dynamics = df_raw_dynamics.rename(columns={'Uniformat Code': 'uniformat_code'})

# using uniformat_code, find the corresponding row in df_raw_dynamics and take the 'Shape Mean' and 'Scale Mean' values
# and add them to df_raw_costs

df_raw_costs['shape'] = df_raw_costs['uniformat_code'].apply(lambda x: df_raw_dynamics[df_raw_dynamics['uniformat_code'] == x]['Shape Mean'].values[0])
df_raw_costs['scale'] = df_raw_costs['uniformat_code'].apply(lambda x: df_raw_dynamics[df_raw_dynamics['uniformat_code'] == x]['Scale Mean'].values[0])

# reorder columns based on replacement cost
#df_raw_costs = df_raw_costs.sort_values(by=['replacement_cost'], ascending=False)
df_raw_costs

# save to csv file
df_raw_costs.to_csv('./components_data_true.csv', index=False)


In [4]:
## load true data and and inspection cost later scale it

building_area_sqft = 9704
budget_per_sqft = 4.54*50
total_budget = building_area_sqft*budget_per_sqft
scaled_budget_min = 0
scaled_budget_max = 10000
inspection_cost_per_sqft = 0.5
total_inspection_cost = building_area_sqft*inspection_cost_per_sqft

df_true_data = pd.read_csv('./components_data_true.csv')

# assign inspection costs
df_true_data['inspection_cost'] = round((df_true_data['replacement_cost'] / df_true_data['replacement_cost'].sum()) * total_inspection_cost,0)

# scale replacement and inspection costs
scaling_factor = (scaled_budget_max - scaled_budget_min) / total_budget
df_true_data['replacement_cost'] = round((df_true_data['replacement_cost']) * scaling_factor,0)
df_true_data['inspection_cost'] = round((df_true_data['inspection_cost']) * scaling_factor,0)

# if inspection cost is less than 1, set it to 1
df_true_data['inspection_cost'] = df_true_data['inspection_cost'].apply(lambda x: 1 if x < 1 else x)

df_true_data = df_true_data.sort_values(by=['replacement_cost'], ascending=False)
df_scaled = df_true_data.iloc[:-2] #ignore last two

# show 15 rows from the middle of the dataframe
#df_scaled = df_true_data.iloc[7:22]

# save to csv file
df_scaled.to_csv('./all_components_data.csv', index=False)

# sum of 15 components replacement cost
df_scaled['replacement_cost'].sum()

1566.0